In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gromo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gromo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.manifold import TSNE

import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from string import punctuation

from gensim.models import KeyedVectors

from sklearn.linear_model import LinearRegression

In [3]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

Посмотрим на пару серпень-август (являющихся переводом)

In [4]:
ru_emb.most_similar([ru_emb["август"]], topn=5)

[('август', 1.0000001192092896),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240029454231262),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447)]

In [5]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999998807907104),
 ('липень', 0.9096441268920898),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992518782615662),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633180618286),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256431221961975)]

In [6]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293082416057587),
 ('офор', 0.23256804049015045),
 ('deteydlya', 0.230317160487175),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608886659145355),
 ('подход', 0.2230587750673294),
 ('иболее', 0.22003726661205292),
 ('2015Александр', 0.21872766315937042),
 ('конструктивен', 0.21796567738056183)]

In [7]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding='utf8') as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)


uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

### Учим маппинг из одного пространства эмбеддингов в другое

У нас есть пары слов, соответствующих друг другу, и их эмбеддинги. Найдем преобразование из одного пространства в другое, чтобы приблизить известные нам слова:

$$W^*= \arg\min_W ||WX - Y||_F, \text{где} ||*||_F - \text{норма Фробениуса}$$

Эта функция очень похожа на линейную регрессию (без биаса).

**Задание** Реализуйте её - воспользуйтесь `LinearRegression` из sklearn с `fit_intercept=False`:

In [8]:
mapping = LinearRegression(fit_intercept=False)
mapping.fit(X_train, Y_train)

LinearRegression(fit_intercept=False)

Проверим, куда перейдет `серпень`:

In [9]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8530694842338562),
 ('июнь', 0.840640127658844),
 ('март', 0.8388842344284058),
 ('сентябрь', 0.8331758975982666),
 ('февраль', 0.831648588180542),
 ('октябрь', 0.8281551003456116),
 ('ноябрь', 0.8248803615570068),
 ('июль', 0.8233261108398438),
 ('август', 0.8108404874801636),
 ('январь', 0.8030163049697876)]

Должно получиться, что в топе содержатся разные месяцы, но август не первый.

Будем мерять percision top-k с k = 1, 5, 10.

**Задание** Реализуйте следующую функцию:

In [10]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = список правильных пар слов [(uk_word_0, ru_word_0), ...]
        mapped_vectors = список эмбедингов после сопоставления исходного пространства эмбедингов в целевое пространство эмбедингов
        topn = количество ближайших соседей в целевом пространстве 
    :returns:
        precision_val, float number, общее количество слов, для которых мы можем найти правильный перевод среди K соседей.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        for j in range(topn):
            if pairs[i][1] in ru_emb.most_similar(mapped_vectors[i].reshape(1, -1), topn=topn)[j][0]:
                num_matches = num_matches+1
                break
            
        
        
        
    precision_val = num_matches / len(pairs)
    return precision_val

In [11]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [12]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [13]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

### Улучшаем маппинг

Можно показать, что маппинг лучше строить ортогональным:
$$W^*= \arg\min_W ||WX - Y||_F \text{, где: } W^TW = I$$

Искать его можно через SVD:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$

$$W^*=UV^T$$

**Задание** Реализуйте эту функцию.

In [14]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    U, s, V = np.linalg.svd(np.matmul(X_train.T, Y_train))
    return np.matmul(U, V)

In [15]:
W = learn_transform(X_train, Y_train)

In [16]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8234362602233887),
 ('июнь', 0.8060609102249146),
 ('сентябрь', 0.8049857020378113),
 ('март', 0.8022727370262146),
 ('октябрь', 0.7987813353538513),
 ('июль', 0.7953117489814758),
 ('ноябрь', 0.7941427826881409),
 ('август', 0.7929527759552002),
 ('февраль', 0.791476845741272),
 ('декабрь', 0.7707280516624451)]

In [17]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

### Пишем переводчик

Реализуем простой пословный переводчик - для каждого слова будем искать его ближайшего соседа в общем пространстве эмбеддингов. Если слова нет в эмбеддингах - просто копируем его.

In [18]:
with open("fairy_tale.txt", "r", encoding="utf-8") as f:
    uk_sentences = [line.rstrip().lower() for line in f]

In [19]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    tr_words = []
    
    for word in sentence.lower().split():
        if word in uk_emb:
            tr_words.append(ru_emb.most_similar([np.matmul(uk_emb[word], W)])[0][0])
        else:
            tr_words.append(word.upper())
        
    return ' '.join(tr_words)
        

In [20]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [21]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк – ПАНІБРАТ

src: як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !
dst: как была себе лисичка , че и пошла раз к обеих бабы добывать огнь ; вошла во избу че и говорит : " хороший день тебе , бабушку !

src: дай мені огня " .
dst: дай мне огня " .

src: а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .
dst: а бабка только что вынула со печи курник со маком , сладкий , че и согнула , чтобы он успокоился ; а лисичка ой и ПІДГЛЕДАЛА , че притом что бабка повалилась во печь , чтобы тварь огня , то лисичка сейчас УХВАТИЛА курник че и деру со хаты , че , пробежать , весь мак со его водица , а туда мус

src: де се ти набрала стільки риби ? "
dst: куда ой ты набрала столько рыбы ? "

src: вона каже : " наловила , вовчику - братику ! "
dst: она говорит : " вывалила , волчонка – братику ! "

src: а собі на думці : " подожди , і я зроблю з тобою таку штуку , як і ти зо мною " .
dst: а себе по мнении : " ветер , и мной сделаю со тобой такую штуку , как и ты оло мной " .

src: - " як же ти ловила ? "
dst: – " как то ты ловила ? "

src: - " так , вовчику , уложила хвостик в ополонку , вожу тихенько да й кажу ; ловися , рибка , мала і велика !
dst: – " так , волчонка , УЛОЖИЛА хвостик во прорубь , вожу тихонько че и говорю ; ЛОВИСЯ , рыбка , имела и большая !

src: коли хочеш , то і ти піди , налови собі " .
dst: когда хочешь , то и ты пойди , НАЛОВИ себе " .

src: він побіг да зробив так , як казала лисичка .
dst: он побежал че сделал так , как говорила лисичка .

src: а лисичка стала за деревом да й дивиться ; коли у вовчика зовсім хвостик примерз , вона тоді побігла в село да й кричить : "